In [52]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
from PIL import Image
from tensorflow.keras.preprocessing import image
import numpy as np

In [53]:
class Callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.99):
            self.model.stop_training = True
            
callback = Callback()

In [54]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(48,48,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
])

In [55]:
model.compile(optimizer = RMSprop(lr=0.001), loss = 'categorical_crossentropy', metrics = ['acc'])

In [56]:
train = ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_generator = train.flow_from_directory(
                    './archive/train',
                    batch_size=64,
                    target_size=(48,48),
                    class_mode='categorical')

Found 24176 images belonging to 5 classes.


In [57]:
validation = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation.flow_from_directory(
                        './archive/test',
                        batch_size=64,
                        target_size=(48,48),
                        class_mode='categorical')

Found 6043 images belonging to 5 classes.


In [59]:
history = model.fit_generator(train_generator,
                              epochs=50,
                              steps_per_epoch=24176//64,
                              verbose=1,
                              validation_data=validation_generator,
                             validation_steps=6043//64,
                             callbacks=[callback])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 377 steps, validate for 94 steps
Epoch 1/50
377/377 [==============================] - 80s 213ms/step - loss: 1.5141 - acc: 0.3223 - val_loss: 1.4437 - val_acc: 0.3740
Epoch 2/50
377/377 [==============================] - 80s 211ms/step - loss: 1.4838 - acc: 0.3460 - val_loss: 1.3898 - val_acc: 0.4087
Epoch 3/50
377/377 [==============================] - 74s 196ms/step - loss: 1.4551 - acc: 0.3632 - val_loss: 1.2992 - val_acc: 0.4639
Epoch 4/50
377/377 [==============================] - 79s 211ms/step - loss: 1.4211 - acc: 0.3887 - val_loss: 1.2759 - val_acc: 0.4619
Epoch 5/50
377/377 [==============================] - 77s 205ms/step - loss: 1.3738 - acc: 0.4151 - val_loss: 1.2649 - val_acc: 0.4830
Epoch 6/50
377/377 [==============================] - 74s 196ms/step - loss: 1.3341 - acc: 0.4394 - val_loss: 1.2231 - val_acc: 0.5048
Epoch 7/50
377/377 [==============================] - 79s 210ms/step - loss: 1.3003 - acc: 0.4589

In [60]:
model.save('prediction_model.h5')